### **Merging rxnorm to dti dataset**


In [1]:
%pip install PyYAML
%pip isntall pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "isntall" - maybe you meant "install"



In [11]:
import pandas as pd
import yaml
import gc

In [3]:
# Load the YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

### Run this code if you dont have the *scope_subset.parquet* file

In [4]:
# pd load
scope_pd = pd.read_parquet(config["paths"]["LINK_TO_DTI_DATASET"])


In [5]:
scope_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208221 entries, 0 to 2208220
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   drug_chembl_id     object 
 1   target_uniprot_id  object 
 2   positive_times     int64  
 3   negative_times     int64  
 4   drug_pubchem_id    float64
 5   label              int64  
 6   smiles             object 
 7   sequence           object 
 8   molfile_3d         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 151.6+ MB


In [8]:
scope_subset = scope_pd[
    [
        "drug_chembl_id",
        "target_uniprot_id",
        "label",
        "smiles",
        "sequence",
        "molfile_3d",
    ]
]


In [9]:
scope_subset.to_parquet("scope_subset.parquet", index=False)

In [ ]:
del scope_subset
gc.collect()

In [13]:
gc.collect()

487

### Merging rxnorm id with scope subset

In [14]:
scope_subset_pd = pd.read_parquet("scope_subset.parquet")

In [15]:
scope_subset_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208221 entries, 0 to 2208220
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   drug_chembl_id     object
 1   target_uniprot_id  object
 2   label              int64 
 3   smiles             object
 4   sequence           object
 5   molfile_3d         object
dtypes: int64(1), object(5)
memory usage: 101.1+ MB


In [16]:
rxcui_chembl_pd = pd.read_parquet(config["paths"]["LINK_TO_RXCUI_DATASET"])

In [17]:
rxcui_chembl_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rxcui        1062 non-null   object 
 1   rxnorm_name  1062 non-null   object 
 2   tty          1062 non-null   object 
 3   pubchem_cid  1045 non-null   float64
 4   inchikey     1062 non-null   object 
 5   drugbank_id  1048 non-null   object 
 6   unii         977 non-null    object 
 7   confidence   1062 non-null   object 
 8   provenance   1045 non-null   object 
 9   chembl_id    1062 non-null   object 
 10  pubchem_id   1062 non-null   Int64  
dtypes: Int64(1), float64(1), object(9)
memory usage: 92.4+ KB


In [18]:
# Merge only common entries (inner join)
scope_onside_common = scope_subset_pd.merge(
    rxcui_chembl_pd[['chembl_id', 'rxcui']], 
    how='inner', 
    left_on='drug_chembl_id', 
    right_on='chembl_id'
)

# Drop duplicate chembl_id column since drug_chembl_id is enough
scope_onside_common = scope_onside_common.drop(columns=['chembl_id'])

In [ ]:
scope_onside_common.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34923 entries, 0 to 34922
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   drug_chembl_id     34923 non-null  object
 1   target_uniprot_id  34923 non-null  object
 2   label              34923 non-null  int64 
 3   smiles             34923 non-null  object
 4   sequence           34923 non-null  object
 5   molfile_3d         34866 non-null  object
 6   rxcui              34923 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.9+ MB


In [21]:
scope_onside_common.to_parquet("../Data/scope_onside_common.parquet", index=False)